## NMD score annotation

In [1]:
import pandas as pd
import re
from tqdm import tqdm
from operator import add

In [36]:
#Import the table with NMD scores

nmdA_df = pd.read_table("/workspace/users/msanchezg/notebooks/NMDetective/nmd.scores.A.tsv.gz", sep='\t')
nmdA_df['alignID.1'] = nmdA_df['transcript_id'].str.split('.',expand=True)[0]
nmdA_df['alignID.1'] = nmdA_df['alignID.1'].str.split(' ',expand=True)[1]
nmdA_df

/home/msanchezg/miniconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (5,8) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,CHROM,Feature,Start,End,Strand,Frame,gene_id,transcript_id,exon_number,exon_id,nmd_score,alignID.1
0,chr1,CDS,69091,70005,+,0,uc001aal.1,uc001aal.1,1,uc001aal.1.1,"-0.09,-0.09,-0.09,-0.09,-0.09,-0.09,-0.09,-0....",uc001aal
1,chr1,CDS,182709,182746,+,0,uc057auo.1,uc057auo.1,1,uc057auo.1.1,"0.23,0.23,0.23,0.24,0.24,0.24,0.24,0.24,0.24,...",uc057auo
2,chr1,CDS,183114,183240,+,1,uc057auo.1,uc057auo.1,2,uc057auo.1.2,"0.31,0.31,0.31,0.31,0.32,0.32,0.32,0.32,0.32,...",uc057auo
3,chr1,CDS,183922,184155,+,0,uc057auo.1,uc057auo.1,3,uc057auo.1.3,"0.17,0.17,0.17,0.17,0.18,0.17,0.17,0.18,0.18,...",uc057auo
4,chr1,CDS,185220,185350,-,2,uc057aup.1,uc057aup.1,1,uc057aup.1.1,"0.1,0.1,0.1,0.07,0.08,0.08,0.08,0.08,0.09,0.1...",uc057aup
...,...,...,...,...,...,...,...,...,...,...,...,...
702477,chrY,CDS,57190204,57190375,+,1,uc004fxm.2,uc004fxm.2,5,uc004fxm.2.5,"0.58,0.58,0.58,0.61,0.61,0.62,0.61,0.61,0.61,...",uc004fxm
702478,chrY,CDS,57190969,57191085,+,0,uc004fxm.2,uc004fxm.2,6,uc004fxm.2.6,"0.61,0.61,0.61,0.61,0.61,0.61,0.61,0.61,0.61,...",uc004fxm
702479,chrY,CDS,57191798,57191999,+,0,uc004fxm.2,uc004fxm.2,7,uc004fxm.2.7,"0.58,0.58,0.58,0.58,0.58,0.58,0.58,0.58,0.57,...",uc004fxm
702480,chrY,CDS,57192603,57192708,+,2,uc004fxm.2,uc004fxm.2,8,uc004fxm.2.8,"0.6,0.65,0.66,0.64,0.59,0.56,0.56,0.54,0.54,0...",uc004fxm


In [19]:
#Import table with UCSC / ENS annotation (UCSC table browser)

annot_df = pd.read_table("/workspace/users/msanchezg/notebooks/NMDetective/annotation.ucsc.ens.tsv.gz", sep="\t")
annot_df['alignID.1'] = annot_df['alignID'].str.split('.',expand=True)[0]
annot_df['Transcript_ens'] = annot_df['#name'].str.split('.',expand=True)[0]
annot_df = annot_df[['Transcript_ens','alignID.1','exonCount']]
annot_df.drop_duplicates(inplace=True, keep='first')
annot_df

,Transcript_ens,alignID.1,exonCount
0,ENST00000456328,uc286dmu,3
1,ENST00000450305,uc286dmv,6
2,ENST00000488147,uc286dmw,11
3,ENST00000619216,uc031tla,1
4,ENST00000473358,uc057aty,3
...,...,...,...
247536,ENST00000615362,uc286tlf,2
247537,ENST00000617983,uc062frj,1
247538,ENST00000613204,uc032quk,5
247539,ENST00000615165,uc032qun,14


In [38]:
#Merge tables

nmdA_annot_df = pd.merge(nmdA_df,annot_df, how='left')
nmdA_annot_df = nmdA_annot_df[['CHROM','Start','End','Strand','Frame','exon_number','exonCount','Transcript_ens','nmd_score']][~nmdA_annot_df['Transcript_ens'].isnull()]
nmdA_annot_df

,CHROM,Start,End,Strand,Frame,exon_number,exonCount,Transcript_ens,nmd_score
0,chr1,69091,70005,+,0,1,3.0,ENST00000641515,"-0.09,-0.09,-0.09,-0.09,-0.09,-0.09,-0.09,-0...."
40,chr1,450743,451678,-,0,1,1.0,ENST00000426406,"0.12,0.12,0.12,0.1,0.11,0.11,0.1,0.11,0.12,0...."
41,chr1,685719,686654,-,0,1,1.0,ENST00000332831,"0.12,0.12,0.12,0.1,0.11,0.11,0.1,0.11,0.12,0...."
42,chr1,925942,926013,+,0,2,7.0,ENST00000420190,"0.3,0.3,0.3,0.3,0.3,0.3,0.3,0.3,0.3,0.29,0.29..."
43,chr1,930155,930336,+,0,3,7.0,ENST00000420190,"0.33,0.33,0.33,0.33,0.33,0.33,0.33,0.33,0.34,..."
...,...,...,...,...,...,...,...,...,...
702477,chrY,57190204,57190375,+,1,5,9.0,ENST00000369423,"0.58,0.58,0.58,0.61,0.61,0.62,0.61,0.61,0.61,..."
702478,chrY,57190969,57191085,+,0,6,9.0,ENST00000369423,"0.61,0.61,0.61,0.61,0.61,0.61,0.61,0.61,0.61,..."
702479,chrY,57191798,57191999,+,0,7,9.0,ENST00000369423,"0.58,0.58,0.58,0.58,0.58,0.58,0.58,0.58,0.57,..."
702480,chrY,57192603,57192708,+,2,8,9.0,ENST00000369423,"0.6,0.65,0.66,0.64,0.59,0.56,0.56,0.54,0.54,0..."


In [ ]:
#Save table for posterior analysis with pyhton/qmap script

nmdA_annot_df.to_csv(r'nmdA.annot.enst.tsv', header=True, index=None, sep='\t')